In [1]:
import numpy as np
import pandas as pd
import nltk

In [2]:
amazon_df=pd.read_csv('amazon_product.csv')

In [3]:
amazon_df.head()

,id,Title,Description,Category
0,1,Swissmar Capstore Select Storage Rack for 18-...,Swissmar's capstore select 18 storage unit kee...,Home & Kitchen Kitchen & Dining Kitchen Utens...
1,2,Gemini200 Delta CV-880 Gold Crown Livery Airc...,Welcome to the exciting world of GeminiJets! O...,Toys & Games Hobbies Models & Model Kits Pre-...
2,5,Superior Threads 10501-2172 Magnifico Cream P...,"For quilting and embroidery, this product is m...","Arts, Crafts & Sewing Sewing Thread & Floss S..."
3,6,Fashion Angels Color Rox Hair Chox Kit,Experiment with the haute trend of hair chalki...,Beauty & Personal Care Hair Care Hair Colorin...
4,8,Union Creative Giant Killing Figure 05: Daisu...,From Union Creative. Turn your display shelf i...,Toys & Games › Action Figures & Statues › Sta...


In [4]:
amazon_df.drop('id',axis=1,inplace=True)

In [5]:
amazon_df.head()

,Title,Description,Category
0,Swissmar Capstore Select Storage Rack for 18-...,Swissmar's capstore select 18 storage unit kee...,Home & Kitchen Kitchen & Dining Kitchen Utens...
1,Gemini200 Delta CV-880 Gold Crown Livery Airc...,Welcome to the exciting world of GeminiJets! O...,Toys & Games Hobbies Models & Model Kits Pre-...
2,Superior Threads 10501-2172 Magnifico Cream P...,"For quilting and embroidery, this product is m...","Arts, Crafts & Sewing Sewing Thread & Floss S..."
3,Fashion Angels Color Rox Hair Chox Kit,Experiment with the haute trend of hair chalki...,Beauty & Personal Care Hair Care Hair Colorin...
4,Union Creative Giant Killing Figure 05: Daisu...,From Union Creative. Turn your display shelf i...,Toys & Games › Action Figures & Statues › Sta...


In [6]:
amazon_df.isnull().sum()

Title          0
Description    0
Category       0
dtype: int64

In [7]:
from nltk.stem.snowball import SnowballStemmer
stemmer=SnowballStemmer('english')
def tokenize_stem(text):
    tokens=nltk.word_tokenize(text.lower())
    stemmed=[stemmer.stem(w) for w in tokens]
    return " ".join(stemmed)

In [8]:
amazon_df['Description']=amazon_df['Description'].apply(tokenize_stem)
amazon_df['Title']=amazon_df['Title'].apply(tokenize_stem)

In [9]:
amazon_df.head()

,Title,Description,Category
0,swissmar capstor select storag rack for 18-pack,swissmar 's capstor select 18 storag unit keep...,Home & Kitchen Kitchen & Dining Kitchen Utens...
1,gemini200 delta cv-880 gold crown liveri aircr...,welcom to the excit world of geminijet ! our m...,Toys & Games Hobbies Models & Model Kits Pre-...
2,superior thread 10501-2172 magnifico cream puf...,"for quilt and embroideri , this product is mad...","Arts, Crafts & Sewing Sewing Thread & Floss S..."
3,fashion angel color rox hair chox kit,experi with the haut trend of hair chalk with ...,Beauty & Personal Care Hair Care Hair Colorin...
4,union creativ giant kill figur 05 : daisuk tsu...,from union creativ . turn your display shelf i...,Toys & Games › Action Figures & Statues › Sta...


In [10]:
amazon_df['stemmed_tokens']=amazon_df['Title']+amazon_df['Description']

In [11]:
amazon_df.head()

,Title,Description,Category,stemmed_tokens
0,swissmar capstor select storag rack for 18-pack,swissmar 's capstor select 18 storag unit keep...,Home & Kitchen Kitchen & Dining Kitchen Utens...,swissmar capstor select storag rack for 18-pac...
1,gemini200 delta cv-880 gold crown liveri aircr...,welcom to the excit world of geminijet ! our m...,Toys & Games Hobbies Models & Model Kits Pre-...,gemini200 delta cv-880 gold crown liveri aircr...
2,superior thread 10501-2172 magnifico cream puf...,"for quilt and embroideri , this product is mad...","Arts, Crafts & Sewing Sewing Thread & Floss S...",superior thread 10501-2172 magnifico cream puf...
3,fashion angel color rox hair chox kit,experi with the haut trend of hair chalk with ...,Beauty & Personal Care Hair Care Hair Colorin...,fashion angel color rox hair chox kitexperi wi...
4,union creativ giant kill figur 05 : daisuk tsu...,from union creativ . turn your display shelf i...,Toys & Games › Action Figures & Statues › Sta...,union creativ giant kill figur 05 : daisuk tsu...


In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf = TfidfVectorizer()

def cosine_sim(txt1, txt2):
    tfidf_matrix = tfidf.fit_transform([txt1, txt2])
    cos_sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])
    return cos_sim[0][0]  # return just the scalar value


In [32]:
def search_product(query):
    stemmed_query=tokenize_stem(query)
    # calculating cosine simalarity between query and stemmed tokens columns
    amazon_df['simalarity']=amazon_df['stemmed_tokens'].apply(lambda x:cosine_sim(stemmed_query,x))
    res=amazon_df.sort_values(by=['simalarity'],ascending=False).head(10)[['Title','Description','Category']]
    return res

In [34]:
search_product('bottle')

,Title,Description,Category
140,"petunia pickl bottom bottl butler , bliss brisban","easy-to-clean and carri along , the bottl butl...",Baby Products › Feeding › Bottle-Feeding › Bo...
263,brita water bottl filter replac - bpa free - 5...,brita water filter bottl replac filter fit all...,Sports & Outdoors Sports & Fitness Accessorie...
517,mrs. meyer liquid hand soap parsley & rhubarb ...,mrs. meyer 's clean day liquid hand soap varie...,"Beauty & Personal Care › Foot, Hand & Nail Ca..."
320,"kalencom lamin buckl bag , rainbow zig zag",a magnet clip keep the top flap close . the po...,Baby Products › Diapering › Diaper Bags
98,"24 , cobalt blue , 15 ml ( 1/2 oz ) glass bott...",these high qualiti cobalt blue glass bottl wit...,Beauty & Personal Care › Tools & Accessories ...
412,"neutrogena fragranc free liquid neutrogena , f...",neutrogena fragranc free liquid facial cleans ...,Beauty & Personal Care Skin Care Face Cleanse...
552,"tervi water bottl , '' cabana with the sun ''",each tumbler featur double-wal insul that keep...,Sports & Outdoors Sports & Fitness Accessorie...
344,"tervi water bottl , life is better at the beach",each tumbler featur double-wal insul that keep...,Sports & Outdoors › Sports & Fitness › Access...
650,"pebeo vitrea 160 , glass paint , 45 ml bottl -...",with vitrea 160 's amaz transpar glass paint i...,"Arts, Crafts & Sewing › Painting, Drawing & A..."
162,"covergirl invis conceal light ( n ) 125 , 0.32...",littl flaw - whi let the world see them ? with...,Beauty & Personal Care › Makeup › Face › Conc...


In [38]:
import nltk

nltk.download('punkt', download_dir='nltk_data')


[nltk_data] Downloading package punkt to nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [41]:
import pickle

with open('model.pkl', 'wb') as file:
    pickle.dump(amazon_df, file)


In [40]:
import numpy as np
import pandas as pd
import nltk
print('numpy',np.__version__)
print('pandas',pd.__version__)
print('nltk',nltk.__version__)

numpy 2.1.3
pandas 2.2.3
nltk 3.9.1
